# Stat542: Linear Regression (I)


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Prepare the Boston Housing Data

In [2]:
%cd ~/Desktop/STAT542_Liang

Boston = pd.read_csv("Data/Boston_Data.csv") 

# Preview the first 5 lines of the loaded data 
Boston.head()

/Users/feng_macpro/Desktop/STAT542_Liang


,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [3]:
Boston.shape

(506, 14)

Check column names

In [ ]:
Boston.columns

Check missing values

In [ ]:
Boston.isnull().sum()

The data frame contains the following columns:

* crim: per capita crime rate by town.
* zn: proportion of residential land zoned for lots over 25,000 sq.ft.
* indus: proportion of non-retail business acres per town.
* chas: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
* nox: nitrogen oxides concentration (parts per 10 million).
* rm: average number of rooms per dwelling.
* age: proportion of owner-occupied units built prior to 1940.
* dis: weighted mean of distances to five Boston employment centres.
* rad: index of accessibility to radial highways.
* tax: full-value property-tax rate per \$10,000.
* ptratio: pupil-teacher ratio by town.
* black: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.
* lstat: lower status of the population (percent).
* __medv__: median value of owner-occupied homes in $1000s.

Change the response variable name to be “Y”. Next take some transformations on Y and X’s, suggested in the literature.

In [6]:
myData = Boston[list(Boston.columns.values)]
myData.columns.values[13] = 'Y'
iLog = ['crim', 'indus', 'nox', 'rm', 'dis', 'rad', 'tax', 'Y']
for i in iLog:
    myData[i] = np.log(myData[i])
myData['zn'] = myData['zn']/10
myData['age'] = np.power(myData['age'], 2.5) / 10000
myData['ptratio'] = np.exp(0.4 * myData['ptratio'])/1000
myData['black'] = myData['black']/100
myData['lstat'] = np.sqrt(myData['lstat'])

A quick summary of each column of myData

In [7]:
myData.describe(include='all')

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,Y
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,-0.780436,1.136364,2.160192,0.069170,-0.610026,1.831864,5.060790,1.188032,1.867661,5.931405,2.150054,3.566740,3.417673,3.034513
std,2.162050,2.332245,0.776987,0.253994,0.201483,0.112325,3.566546,0.539547,0.874833,0.396367,1.364011,0.912949,0.987167,0.408757
min,-5.064036,0.000000,-0.776529,0.000000,-0.954512,1.270041,0.001432,0.121864,0.000000,5.231109,0.154470,0.003200,1.315295,1.609438
25%,-2.500488,0.000000,1.646734,0.000000,-0.800732,1.772492,1.360301,0.742021,1.386294,5.631212,1.053634,3.753775,2.636277,2.834680
50%,-1.360641,0.000000,2.271094,0.000000,-0.619897,1.825919,5.287613,1.165473,1.609438,5.799093,2.038970,3.914400,3.370459,3.054001
75%,1.302119,1.250000,2.895912,0.000000,-0.471605,1.890624,8.583922,1.646399,3.178054,6.501290,3.229233,3.962250,4.117645,3.218876
max,4.488369,10.000000,3.322875,1.000000,-0.138113,2.172476,10.000000,2.495393,3.178054,6.566672,6.634244,3.969000,6.161980,3.912023


Produce a pair-wise scatter plot. Caution: a big figure.

In [ ]:
sns.pairplot(myData)

### Fit a Linear Model

Next we describe how to fit a linear regression model in Python and how to obtain the LS prediction on new samples. 

Our new samples are two houses with median features (i.e., all covariates except `chas` take the median value of the 506 samples), and one is close to the Charles River and the other one is not.

In [9]:
Xnew = pd.concat([myData.median()] * 2, axis = 1)
Xnew = pd.DataFrame(Xnew).transpose()
Xnew['chas'] = [0, 1]
Xnew = Xnew.iloc[:, :-1]
Xnew

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,-1.360641,0.0,2.271094,0,-0.619897,1.825919,5.287613,1.165473,1.609438,5.799093,2.03897,3.9144,3.370459
1,-1.360641,0.0,2.271094,1,-0.619897,1.825919,5.287613,1.165473,1.609438,5.799093,2.03897,3.9144,3.370459


We consider __Three__ different ways to fit a linear regression model in Python: 

1. Use `sklearn.linear_model`

In [ ]:
from sklearn.linear_model import LinearRegression as lm
X = myData.iloc[:, :-1]
lmfit1 = lm()
lmfit1.fit(X, myData['Y'])
print(lmfit1.coef_)
print(lmfit1.intercept_)

In [11]:
print(lmfit1.predict(Xnew), lmfit1.intercept_ + np.inner(Xnew, lmfit1.coef_)) 

[3.07962244 3.18960258] [3.07962244 3.18960258]


2. Use `statsmodels.formula.api`. You can use R-style formulas to specific models, but you cannot use `Y ~ . ` and have to list all the covariates included in the model. 

In [12]:
import statsmodels.formula.api as smf
all_covariates = list(myData.columns)
all_covariates.remove('Y')
my_formula = "Y ~ " + "+".join(all_covariates)
lmfit2 = smf.ols(formula = my_formula, data = myData).fit()
lmfit2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.759
Method:                 Least Squares   F-statistic:                     123.2
Date:                Tue, 17 Sep 2019   Prob (F-statistic):          2.33e-145
Time:                        00:51:42   Log-Likelihood:                 101.59
No. Observations:                 506   AIC:                            -175.2
Df Residuals:                     492   BIC:                            -116.0
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.1769      0.379     11.020      0.000       3.432       4.922
crim          -0.0146      0.012     -1.254      0.211      -0.037       0.008
zn             0.0014      0.006      0.247      0.805      -0.010       0.012
indus         -0.0127      0.022     -0.570      0.569      -0.057       0.031
chas           0.1100      0.037      3.002      0.003       0.038       0.182
nox           -0.2831      0.105     -2.688      0.007      -0.490      -0.076
rm             0.4211      0.110      3.822      0.000       0.205       0.638
age            0.0064      0.005      1.317      0.189      -0.003       0.016
dis           -0.1832      0.037     -4.977      0.000      -0.255      -0.111
rad            0.0684      0.022      3.042      0.002       0.024       0.113
tax           -0.2018      0.048     -4.167      0.000      -0.297      -0.107
ptratio       -0.0400      0.008     -4.946      0.000      -0.056      -0.024
black          0.0445      0.011      3.882      0.000       0.022       0.067
lstat         -0.2626      0.016    -16.320      0.000      -0.294      -0.231
==============================================================================
Omnibus:                       50.620   Durbin-Watson:                   1.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              240.340
Skew:                          -0.257   Prob(JB):                     6.47e-53
Kurtosis:                       6.337   Cond. No.                         458.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
lmfit2.predict(Xnew)

0    3.079622
1    3.189603
dtype: float64

In [14]:
lmfit2.params[0] + np.inner(Xnew, lmfit2.params[1:])

array([3.07962244, 3.18960258])

3. Use `statsmodels.api`. When using `statsmodels.formula.api`, like in R, the intercept will be automaticlaly added to the model, but this is not the case with `statsmodels.api`. 

In [ ]:
import statsmodels.api as sm
X = myData.iloc[:, :-1]
X = sm.add_constant(X) # need to add the constant in the design matrix
X.head()
lmfit3 = sm.OLS(myData['Y'], X).fit()
lmfit3.summary()

To use the built-in prediction function for `sm.OLS`, we need to add a constant term to Xnew. However, Xnew already has some constant columns (i.e., columns of identical values); by default, `sm.add_constant` won't add a constant column to Xnew. So we need to use the option `has_constant='add'`

In [ ]:
Xnew_const = sm.add_constant(Xnew, has_constant='add')
lmfit3.predict(Xnew_const)

In [ ]:
np.inner(Xnew_const, lmfit3.params)

### How to Compute Some Regression Outputs

Next, let's check how residual standard error, Log-Likelihood, and R-square are computed using the output from the second approach `lmfit2`

In [18]:
residuals = myData['Y'] - lmfit2.predict()
len(residuals)

506

In [19]:
sigma_hat = np.sqrt(np.sum(residuals**2)/(506 - 14)) # Residual Standard Error
sigma_hat

0.20075284297263912

In [20]:
-(506/2)*np.log(2*np.pi*(sigma_hat**2)) - (506-14)/2 # Log-Likelihood

101.49156903489637

In [21]:
1 - np.sum(residuals**2)/(myData.loc[:, 'Y'].var()*505)  # R-square

0.7650004422627311

### Rank Deficiency

If the design matrix (including the intercept) is not of full rank, as we have learned in class, the LS coefficient vector is not unique. 

In the following example, we add a redudant column variable `junk` to `myData`, and then fit a linear model using `statsmodels.formula.api` and `sklearn.linear_model`. Python will return just one of many equivalent LS solutions along with a warning on design matrix being singular. You can still use the fitted model to do prediction. The prediction result is the same as before. 

In [22]:
myData['junk'] = myData['crim'] + myData['zn']
my_formula = my_formula + '+ junk'
tmpfit2 = smf.ols(formula = my_formula, data = myData).fit()
tmpfit2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.759
Method:                 Least Squares   F-statistic:                     123.2
Date:                Tue, 17 Sep 2019   Prob (F-statistic):          2.33e-145
Time:                        00:51:43   Log-Likelihood:                 101.59
No. Observations:                 506   AIC:                            -175.2
Df Residuals:                     492   BIC:                            -116.0
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.1769      0.379     11.020      0.000       3.432       4.922
crim          -0.0102      0.008     -1.303      0.193      -0.026       0.005
zn             0.0058      0.005      1.122      0.262      -0.004       0.016
indus         -0.0127      0.022     -0.570      0.569      -0.057       0.031
chas           0.1100      0.037      3.002      0.003       0.038       0.182
nox           -0.2831      0.105     -2.688      0.007      -0.490      -0.076
rm             0.4211      0.110      3.822      0.000       0.205       0.638
age            0.0064      0.005      1.317      0.189      -0.003       0.016
dis           -0.1832      0.037     -4.977      0.000      -0.255      -0.111
rad            0.0684      0.022      3.042      0.002       0.024       0.113
tax           -0.2018      0.048     -4.167      0.000      -0.297      -0.107
ptratio       -0.0400      0.008     -4.946      0.000      -0.056      -0.024
black          0.0445      0.011      3.882      0.000       0.022       0.067
lstat         -0.2626      0.016    -16.320      0.000      -0.294      -0.231
junk          -0.0044      0.004     -0.988      0.324      -0.013       0.004
==============================================================================
Omnibus:                       50.620   Durbin-Watson:                   1.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              240.340
Skew:                          -0.257   Prob(JB):                     6.47e-53
Kurtosis:                       6.337   Cond. No.                     1.62e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.13e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [23]:
Xnew['junk'] = Xnew['crim'] + X['zn']
tmpfit2.predict(Xnew)

0    3.071694
1    3.189603
dtype: float64

In [24]:
X = myData.drop(['Y'], axis=1)
tmpfit1 = lm()
tmpfit1.fit(X, myData['Y'])
print(tmpfit1.coef_)
print(tmpfit1.intercept_)

[-0.01020156  0.00579675 -0.01270937  0.10998014 -0.28311188  0.42110784
  0.00640337 -0.18315429  0.06836159 -0.20183238 -0.04001744  0.04447193
 -0.26261509 -0.00440481]
4.176874035061328


In [25]:
tmpfit1.predict(Xnew)

array([3.07169378, 3.18960258])